# CRUD Operations

This notebook demonstrates all CRUD operations in pandalchemy:
- **C**reate: add_row, bulk_insert
- **R**ead: get_row, row_exists
- **U**pdate: update_row, upsert_row
- **D**elete: delete_row

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandalchemy as pa

## Setup
Create a database and initial table with auto-increment

In [2]:
engine = create_engine('sqlite:///:memory:')
db = pa.DataBase(engine)

# Create initial table
initial_data = pd.DataFrame({
    'name': ['Alice', 'Bob'],
    'email': ['alice@example.com', 'bob@example.com'],
    'age': [25, 30]
}, index=[1, 2])

users = pa.TableDataFrame('users', initial_data, 'id', engine, auto_increment=True)
users.push()

users.to_pandas()

,name,email,age
id,,,
1,Alice,alice@example.com,25
2,Bob,bob@example.com,30


## 1. CREATE Operations

### Add single row with auto-increment

In [3]:
users.add_row({'name': 'Charlie', 'email': 'charlie@example.com', 'age': 35}, auto_increment=True)
print("✓ Added Charlie (ID will be auto-generated)")

✓ Added Charlie (ID will be auto-generated)


### Bulk insert multiple rows

In [4]:
new_users = [
    {'name': 'Diana', 'email': 'diana@example.com', 'age': 28},
    {'name': 'Eve', 'email': 'eve@example.com', 'age': 32},
    {'name': 'Frank', 'email': 'frank@example.com', 'age': 45}
]

for user in new_users:
    users.add_row(user, auto_increment=True)

users.push()
print(f"✓ Added {len(new_users)} users")
print(f"Total users: {len(users._data)}")

users.to_pandas()

✓ Added 3 users
Total users: 6


,name,email,age
id,,,
1,Alice,alice@example.com,25
2,Bob,bob@example.com,30
3,Charlie,charlie@example.com,35
4,Diana,diana@example.com,28
5,Eve,eve@example.com,32
6,Frank,frank@example.com,45


## 2. READ Operations

### Get row by primary key

In [5]:
users.get_row(1)

{'name': 'Alice', 'email': 'alice@example.com', 'age': 25}

### Check if row exists

In [6]:
print(f"User 1 exists: {users.row_exists(1)}")
print(f"User 999 exists: {users.row_exists(999)}")

User 1 exists: True
User 999 exists: False


### Get all data

In [7]:
df = users.to_pandas()
print(f"Total rows: {len(df)}, Columns: {list(df.columns)}")
df

Total rows: 6, Columns: ['name', 'email', 'age']


,name,email,age
id,,,
1,Alice,alice@example.com,25
2,Bob,bob@example.com,30
3,Charlie,charlie@example.com,35
4,Diana,diana@example.com,28
5,Eve,eve@example.com,32
6,Frank,frank@example.com,45


### Filter data using pandas

In [8]:
seniors = users._data[users._data['age'] > 30]
print(f"Users over 30: {len(seniors)}")
seniors[['name', 'age']]

Users over 30: 3


,name,age
id,,
3,Charlie,35
5,Eve,32
6,Frank,45


## 3. UPDATE Operations

### Update single field

In [9]:
users.update_row(1, {'age': 26})
print("✓ Updated Alice's age to 26")

✓ Updated Alice's age to 26


### Update multiple fields

In [10]:
users.update_row(2, {'email': 'robert@example.com', 'age': 31})
print("✓ Updated Bob's email and age")

✓ Updated Bob's email and age


### Upsert operation (update or insert)

In [11]:
# Update existing
users.upsert_row({'id': 2, 'name': 'Robert', 'email': 'robert@example.com', 'age': 31})
print("✓ Upserted user 2 (updated existing)")

# Insert new
users.upsert_row({'id': 100, 'name': 'Grace', 'email': 'grace@example.com', 'age': 29})
print("✓ Upserted user 100 (created new)")

users.push()
users.to_pandas()

✓ Upserted user 2 (updated existing)
✓ Upserted user 100 (created new)


,name,email,age
id,,,
1,Alice,alice@example.com,26
2,Robert,robert@example.com,31
3,Charlie,charlie@example.com,35
4,Diana,diana@example.com,28
5,Eve,eve@example.com,32
6,Frank,frank@example.com,45
100,Grace,grace@example.com,29


## 4. DELETE Operations

### Delete rows

In [12]:
users.delete_row(100)
users.delete_row(5)
users.delete_row(6)

users.push()
print(f"✓ Remaining users: {len(users._data)}")
users.to_pandas()

✓ Remaining users: 4


,name,email,age
id,,,
1,Alice,alice@example.com,26
2,Robert,robert@example.com,31
3,Charlie,charlie@example.com,35
4,Diana,diana@example.com,28


## 5. Pandas Integration
All pandas operations work seamlessly with pandalchemy

In [13]:
# Update via pandas .loc and .at
users._data.loc[1, 'email'] = 'alice.updated@example.com'
users._data.at[2, 'age'] = 32

# Add column via pandas
users._data['verified'] = [True, False, True, False]

users.push()
users.to_pandas()

,name,email,age
id,,,
1,Alice,alice.updated@example.com,26
2,Robert,robert@example.com,32
3,Charlie,charlie@example.com,35
4,Diana,diana@example.com,28


## 6. Error Handling

In [14]:
# Try to update non-existent row
try:
    users.update_row(999, {'age': 100})
except Exception as e:
    print(f"✓ Correctly raised: {type(e).__name__}")

# Try to update primary key (immutable)
from pandalchemy.exceptions import DataValidationError
try:
    users.update_row(1, {'id': 999})
except DataValidationError:
    print("✓ Correctly prevented: Primary keys are immutable")

✓ Correctly raised: ValueError
✓ Correctly prevented: Primary keys are immutable


## Summary

**Key Takeaways:**
- `add_row()` with `auto_increment=True` for automatic IDs
- `get_row()` and `row_exists()` for reading data
- `update_row()` for updates, `upsert_row()` for update-or-insert
- `delete_row()` for deletions
- All pandas DataFrame operations work and are automatically tracked
- Primary keys are immutable (stored as DataFrame index)